In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from glob import glob
import os
import data_util

In [25]:
device = torch.device('cuda')
dtype = torch.float32

In [26]:
label_str = ['bench','cabinet','car','chair','gun','light','phone','plane','screen','ship','sofa','stereo','table']
str2num = {label_str[i]:i for i in range(len(label_str))}

In [27]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, train=True):
        super().__init__()
        self.train = train
        if train == True:
            self.all_images = sorted(glob(os.path.join(root_dir, 'train_rgb', '*.png')))
        if train == False:
            self.all_images = sorted(glob(os.path.join(root_dir, 'test_rgb', '*.png')))
            
    def __len__(self):
        return len(self.all_images)        
    
    def __getitem__(self, idx):
        image_dir = self.all_images[idx]
        image = data_util.load_img(image_dir, square_crop=True, downsampling_order=1, target_size=[256, 256])
        image = image[:, :, :3].astype(np.float32) / 255. - 0.5
        image = image.transpose(2,0,1)
        image = torch.from_numpy(image)
        
        label = str2num[image_dir.split('/')[-1].split('_')[0]]
        
        return (image, label)

In [28]:
train_dataset = Dataset(os.path.join('/home/max/classification_dataset'), True)
train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_dataset = Dataset(os.path.join('/home/max/classification_dataset'), False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [29]:
def train(model, optimizer, epochs=1):

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x,y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % 20 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                
        torch.save(model.state_dict(), os.path.join('/home/max/saved_data/classifier2/model_epoch'+str(e)+'.pth'))

In [30]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
    
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 3, padding = 1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size = 3, padding = 1)
        self.conv3 = nn.Conv2d(32, 32, kernel_size = 3, padding = 1)
        self.conv4 = nn.Conv2d(32, 32, kernel_size = 3, padding = 1)
        self.conv5 = nn.Conv2d(32, 32, kernel_size = 3, padding = 1)
        self.fc1 = nn.Linear(32*8*8, 64)
        self.fc2 = nn.Linear(64, 13)
        self.maxpool = nn.MaxPool2d(2, stride=2, padding=0)
        
        
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.maxpool(out)
        out = self.conv2(out)
        out = F.relu(out)
        out = self.maxpool(out)
        out = self.conv3(out)
        out = F.relu(out)
        out = self.maxpool(out)
        out = self.conv4(out)
        out = F.relu(out)
        out = self.maxpool(out)
        out = self.conv5(out)
        out = F.relu(out)
        out = self.maxpool(out)
        
        out = out.view(out.shape[0],-1)
        out = self.fc1(out)
        out = F.relu(out)
        scores = self.fc2(out)
        return scores

In [31]:
learning_rate = 1e-3
model = ConvNet()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


train(model, optimizer, epochs=30)

Iteration 0, loss = 2.6312
Iteration 20, loss = 2.5659
Iteration 40, loss = 2.5343
Iteration 60, loss = 2.5889
Iteration 80, loss = 2.5087
Iteration 100, loss = 2.5262
Iteration 120, loss = 2.5913
Iteration 140, loss = 2.5352
Iteration 160, loss = 2.5523
Iteration 180, loss = 2.4088
Iteration 200, loss = 2.5669
Iteration 220, loss = 2.4198
Iteration 240, loss = 1.5887
Iteration 260, loss = 2.6147
Iteration 280, loss = 2.2683
Iteration 300, loss = 1.9307
Iteration 320, loss = 1.7122
Iteration 340, loss = 1.3170
Iteration 360, loss = 1.8489
Iteration 380, loss = 1.0947
Iteration 400, loss = 1.2103
Iteration 420, loss = 2.5032
Iteration 440, loss = 1.1465
Iteration 460, loss = 1.5404
Iteration 480, loss = 1.0071
Iteration 500, loss = 1.4545
Iteration 520, loss = 1.4662
Iteration 540, loss = 1.2535
Iteration 560, loss = 1.4909
Iteration 580, loss = 1.5115
Iteration 600, loss = 1.2997
Iteration 620, loss = 1.4619
Iteration 640, loss = 1.4997
Iteration 660, loss = 0.8429
Iteration 680, loss 

Iteration 640, loss = 0.3809
Iteration 660, loss = 0.2236
Iteration 680, loss = 0.0487
Iteration 700, loss = 0.2145
Iteration 720, loss = 0.5467
Iteration 740, loss = 0.4188
Iteration 760, loss = 0.7424
Iteration 780, loss = 0.0556
Iteration 800, loss = 0.1578
Iteration 820, loss = 0.1488
Iteration 0, loss = 0.1497
Iteration 20, loss = 0.2725
Iteration 40, loss = 0.0006
Iteration 60, loss = 0.1483
Iteration 80, loss = 0.4220
Iteration 100, loss = 0.2539
Iteration 120, loss = 1.1014
Iteration 140, loss = 0.7071
Iteration 160, loss = 0.3934
Iteration 180, loss = 0.5646
Iteration 200, loss = 0.2844
Iteration 220, loss = 0.0048
Iteration 240, loss = 0.0188
Iteration 260, loss = 1.1212
Iteration 280, loss = 0.0856
Iteration 300, loss = 0.0191
Iteration 320, loss = 0.4627
Iteration 340, loss = 0.7126
Iteration 360, loss = 0.4704
Iteration 380, loss = 0.1070
Iteration 400, loss = 0.3543
Iteration 420, loss = 0.8160
Iteration 440, loss = 0.6449
Iteration 460, loss = 0.2950
Iteration 480, loss 

Iteration 440, loss = 0.0021
Iteration 460, loss = 0.0697
Iteration 480, loss = 0.0016
Iteration 500, loss = 0.0009
Iteration 520, loss = 0.1520
Iteration 540, loss = 0.2117
Iteration 560, loss = 0.0023
Iteration 580, loss = 0.0065
Iteration 600, loss = 0.0082
Iteration 620, loss = 0.1918
Iteration 640, loss = 0.0168
Iteration 660, loss = 0.0095
Iteration 680, loss = 0.0207
Iteration 700, loss = 0.1416
Iteration 720, loss = 0.0373
Iteration 740, loss = 0.0653
Iteration 760, loss = 0.0253
Iteration 780, loss = 0.0005
Iteration 800, loss = 0.2532
Iteration 820, loss = 0.0022
Iteration 0, loss = 0.0121
Iteration 20, loss = 0.0119
Iteration 40, loss = 0.2507
Iteration 60, loss = 0.9958
Iteration 80, loss = 0.1711
Iteration 100, loss = 0.0049
Iteration 120, loss = 0.0167
Iteration 140, loss = 0.0001
Iteration 160, loss = 0.0376
Iteration 180, loss = 0.0000
Iteration 200, loss = 0.0001
Iteration 220, loss = 0.0083
Iteration 240, loss = 0.0912
Iteration 260, loss = 0.2278
Iteration 280, loss 

Iteration 240, loss = 0.0147
Iteration 260, loss = 0.0322
Iteration 280, loss = 0.0004
Iteration 300, loss = 0.0000
Iteration 320, loss = 0.0002
Iteration 340, loss = 0.0080
Iteration 360, loss = 0.7186
Iteration 380, loss = 0.0000
Iteration 400, loss = 0.2053
Iteration 420, loss = 0.0324
Iteration 440, loss = 0.0009
Iteration 460, loss = 0.0188
Iteration 480, loss = 0.0050
Iteration 500, loss = 0.0053
Iteration 520, loss = 0.0073
Iteration 540, loss = 0.0032
Iteration 560, loss = 0.0671
Iteration 580, loss = 0.0031
Iteration 600, loss = 0.0000
Iteration 620, loss = 0.2548
Iteration 640, loss = 0.3109
Iteration 660, loss = 0.4623
Iteration 680, loss = 0.0000
Iteration 700, loss = 0.0036
Iteration 720, loss = 0.0470
Iteration 740, loss = 0.2274
Iteration 760, loss = 0.1776
Iteration 780, loss = 0.0000
Iteration 800, loss = 0.6147
Iteration 820, loss = 0.0581
Iteration 0, loss = 0.0001
Iteration 20, loss = 0.0091
Iteration 40, loss = 0.0001
Iteration 60, loss = 0.0383
Iteration 80, loss 

Iteration 40, loss = 0.6546
Iteration 60, loss = 0.0000
Iteration 80, loss = 0.0005
Iteration 100, loss = 0.0000
Iteration 120, loss = 0.0092
Iteration 140, loss = 0.0004
Iteration 160, loss = 0.3651
Iteration 180, loss = 0.0005
Iteration 200, loss = 0.1801
Iteration 220, loss = 0.0007
Iteration 240, loss = 0.0017
Iteration 260, loss = 0.0001
Iteration 280, loss = 0.0001
Iteration 300, loss = 0.0119
Iteration 320, loss = 0.0000
Iteration 340, loss = 0.0076
Iteration 360, loss = 0.0000
Iteration 380, loss = 0.0005
Iteration 400, loss = 0.0029
Iteration 420, loss = 0.0040
Iteration 440, loss = 0.0249
Iteration 460, loss = 0.0001
Iteration 480, loss = 0.0009
Iteration 500, loss = 0.0002
Iteration 520, loss = 0.0941
Iteration 540, loss = 0.0034
Iteration 560, loss = 0.0006
Iteration 580, loss = 0.0001
Iteration 600, loss = 0.0000
Iteration 620, loss = 0.0217
Iteration 640, loss = 0.0003
Iteration 660, loss = 0.0029
Iteration 680, loss = 0.0013
Iteration 700, loss = 0.0000
Iteration 720, lo

In [32]:
def check_accuracy_part(loader, model): 
    num_correct = 0
    num_samples = 0
    model = model.to(device=device)
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
        return acc

In [33]:
accs = list()
for i in range(0,30,2):
    model = ConvNet()
    model.load_state_dict(torch.load(os.path.join('/home/max/saved_data/classifier2/model_epoch'+str(i)+'.pth')))
    acc = check_accuracy_part(test_loader, model)
    accs.append(acc)

accs = np.array(accs)

Got 611 / 1040 correct (58.75)
Got 701 / 1040 correct (67.40)
Got 710 / 1040 correct (68.27)
Got 696 / 1040 correct (66.92)
Got 710 / 1040 correct (68.27)
Got 686 / 1040 correct (65.96)
Got 715 / 1040 correct (68.75)
Got 693 / 1040 correct (66.63)
Got 694 / 1040 correct (66.73)
Got 672 / 1040 correct (64.62)
Got 688 / 1040 correct (66.15)
Got 685 / 1040 correct (65.87)
Got 662 / 1040 correct (63.65)
Got 696 / 1040 correct (66.92)
Got 676 / 1040 correct (65.00)


In [34]:
np.save(os.path.join('/home/max/saved_data/classifier2'),accs)